In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.onnx as torch_onnx

class Customize_Model(nn.Module):
    def __init__(self, model_name, cls):
        super().__init__()
        
    def forward(self, image):
        x = self.model(image)
        return x

input_shape = (3, 128, 313)
# input_shape = (3, 192, 470)

# torch_model_path= './train_model/cv0_best.pth'
torch_model_path= './test_model/effb0_imgsz128_PL2/model_soup.pth'

model= torch.load(torch_model_path, map_location='cpu').eval()

# dynamic input: can accept different input size image
model_onnx_path = torch_model_path.replace('pth', 'onnx')

# Export the model to an ONNX file
dummy_input = Variable(torch.randn(1, *input_shape))
output = torch_onnx.export(model, 
                           dummy_input, 
                           model_onnx_path, 
#                            opset_version=12,
                           input_names= ['input'],
                           verbose=False)

C:\Users\User\anaconda3\envs\kaggle_smbcd\lib\site-packages\timm\layers\padding.py:19: TracerWarning: Converting a tensor to a Python float might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  return max((math.ceil(x / s) - 1) * s + (k - 1) * d + 1 - x, 0)
C:\Users\User\anaconda3\envs\kaggle_smbcd\lib\site-packages\timm\layers\padding.py:19: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  return max((math.ceil(x / s) - 1) * s + (k - 1) * d + 1 - x, 0)
C:\Users\User\anaconda3\envs\kaggle_smbcd\lib\site-packages\timm\layers\padding.py:31: TracerWarning: Converting a tensor to a Python boolean might cause the trace to

============= Diagnostic Run torch.onnx.export version 2.0.0+cu117 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [2]:
import onnxruntime

session = onnxruntime.InferenceSession(torch_model_path.replace('pth', 'onnx'),
                                       providers=[
#                                             'TensorrtExecutionProvider',
#                                             'CUDAExecutionProvider',
                                            'CPUExecutionProvider'
                                       ])

In [3]:
import onnxruntime
import numpy as np
from PIL import Image
import time

def model_predict(session, img):
    
    session.get_modelmeta()
    first_input_name = session.get_inputs()[0]
    first_output_name = session.get_outputs()[0]

    inname = [input.name for input in session.get_inputs()]
    outname = [output.name for output in session.get_outputs()]

    data_output = session.run(outname, {inname[0]: img})

    pred= {}
    pred['label']= data_output[0]
    return pred

In [4]:
import numpy as np
import torch
from PIL import Image
import cv2

img= torch.rand(1,3,128,313)

model.eval()
with torch.no_grad():
    pred= model(img)
print(pred[0][:10])

pred= model_predict(session, img.numpy())['label']
print(pred[0][:10])

tensor([5.4291, 5.7834, 5.3119, 5.6427, 5.8332, 4.8138, 4.5394, 4.4131, 6.1539,
        4.7093])
[5.4290733 5.783387  5.311868  5.6426854 5.8332057 4.813763  4.5394173
 4.413128  6.153949  4.7093315]
